In [1]:
import pandas as pd
import os
import sys
import MeCab

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
#出現の割合の下限値を入力すると，それに満たない語を出力
def stop_word_min(corpus, num):
    count_vectorizer = CountVectorizer(max_df=num)
    X_count = count_vectorizer.fit_transform(corpus)
    
    return count_vectorizer.get_feature_names()

In [3]:
#出現の割合の上限値を入力すると，それを超える語を出力
def stop_word_max(corpus, num):
    count_vectorizer = CountVectorizer(min_df=num)
    X_count = count_vectorizer.fit_transform(corpus)
    
    return count_vectorizer.get_feature_names()

In [4]:
review_df = pd.read_csv("1005data.csv")
review_df = review_df.rename(columns={'Unnamed: 0':'index', 'title':'title', 'rate':'score', 'reviews':'review'}) 
review_df

,index,id,Unnamed: 0.1,title,score,review
0,0,0,0,アナと雪の女王2,3.87,['\n正直1作目のアナ雪は好きではなかった。もちろん音楽と映像は素晴らしかったしキャラクタ...
1,1,1,1,ドクター・スリープ,3.52,['\nあの傑作ホラー、1980年「シャイニング」を劇場公開で観た スタンリー・キューブリッ...
2,2,2,2,映画 すみっコぐらし とびだす絵本とひみつのコ,4.43,['\n大人受けした理由は明らかだ。子供向けの映画を作ったのではなく「すみっコの物語」を作っ...
3,3,3,3,ターミネーター：ニュー・フェイト,3.69,['\n海外にて鑑賞。新作が公開されるたび期待と不安が入り混じるこのシリーズ。目玉である「T...
4,4,4,4,シティーハンター THE MOVIE 史上最香のミッション,4.21,['\nファンから観たCH（公開前なので一切ネタバレないよう書きます）大分前にフランス版を鑑...
...,...,...,...,...,...,...
998,998,1000,475,THE 有頂天ホテル,3.36,['\u3000とても良く出来てはいるのですが、大爆笑とか深い感動とかを期待するのもどうかと...
999,999,1001,476,パージ：アナーキー,3.46,"[""『パージ：アナーキー』(THE PURGE：ANARCHY／2014／米)は､前作同様ジ..."
1000,1000,1002,477,Dear Friends ディア フレンズ,3.41,['Yoshiの原作がひどいのか､それとも脚本と監督がひどいのかはわからないが､まさしく少女...
1001,1001,1003,478,エージェント・ウルトラ,3.07,['予告編に騙されることが多いとです（笑）最近も「人生の約束」はがっかりでしたが、今回も見事...


In [5]:
def analyze(text):
    tagger = MeCab.Tagger("-Ochasen")
    tagger.parse('')
    node = tagger.parseToNode(text)
    wakati = ''
    while node:
        word = node.surface
        wclass = node.feature.split(',')
        #mecabは以下の順でnode.featureに情報を入れる　[0]品詞,[1]品詞細分類1,[2]品詞細分類2,[3]品詞細分類3,[4]活用形,[5]活用型,[6]原形,[7]読み,[8]発音
        if (
            #wclass[0] == '記号' or
            wclass[0] == '形容詞' or
            #wclass[0] == '助詞' or
            #wclass[0] == '助動詞' or
            #wclass[0] == '接続詞' or
            #wclass[0] == '接頭詞' or
            wclass[0] == '動詞' or
            #wclass[0] == '副詞' or 
            wclass[0] == '名詞'
            #wclass[0] == '連体詞'
        ):
            if wclass[6] != '*':
                wakati += wclass[6] + ' '
        node = node.next
    return wakati


In [6]:
def make_data(pd):
    training = []
    for index in pd.index:
        sent = analyze(pd.at[index,'review'])
        training.append(sent)
    
    return training

In [14]:
swmin=stop_word_min(make_data(review_df),0.001)

In [18]:
swmax=stop_word_max(make_data(review_df),0.50)

In [16]:
len(swmin)

9782

In [19]:
len(swmax)

246